In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192",
    temperature=0.7
)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [2]:
#To load pdf files

pdf_folder_path = "./Data/RagFiles"

loader = DirectoryLoader(
    pdf_folder_path,
    glob="*.pdf",
    loader_cls=PyPDFLoader
)
raw_docs = loader.load()

print(f"Loaded {len(raw_docs)} PDF documents from {pdf_folder_path}")


Loaded 81 PDF documents from ./Data/RagFiles


In [3]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = splitter.split_documents(raw_docs)
print(f"After splitting, we have {len(docs)} chunks total.")


After splitting, we have 338 chunks total.


In [4]:
class SentenceTransformerEmbeddings:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

embedding_func = SentenceTransformerEmbeddings()

db = Chroma.from_documents(docs, embedding=embedding_func, persist_directory="./chroma_db")

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [5]:
@tool
def add(input_str: str) -> str:
    """Add two numbers from a string like 1, 2."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a + b)

@tool
def subtract(input_str: str) -> str:
    """Subtract two numbers from a string like 5, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a - b)

@tool
def multiply(input_str: str) -> str:
    """Multiply two numbers from a string like 2, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a * b)

@tool
def divide(input_str: str) -> str:
    """Divide two numbers from a string like 6, 2."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    if b == 0:
        return "Error: Division by zero"
    return str(a / b)

@tool
def power(input_str: str) -> str:
    """Raise one number to another from a string like 2, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a ** b)

@tool
def modulus(input_str: str) -> str:
    """Compute the remainder for 'a mod b' from a string like 5, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a % b)
@tool
def web_search(query: str) -> str:
    """
    Search the web using SerpAPI for the given query.
    Returns a summary of the top results.
    """
    try:
        serpapi = SerpAPIWrapper()
        return serpapi.run(query)
    except Exception as e:
        return f"Error searching: {str(e)}"

@tool
def pdf_rag(query: str) -> str:
    """
    Retrieve relevant PDF content based on the query.
    
    Args:
        query (str): Search query to find relevant PDF content
    
    Returns:
        str: Chunks of relevant PDF content
    """

    results = retriever.get_relevant_documents(query)
    
    if not results:
        return f"No relevant documents found for query: {query}"

    return results

@tool
def summarize_file(query: str) -> str:
    """
    Summarizes PDF files related to the given query.
    
    Args:
        query (str): Search query to find relevant PDF files
    
    Returns:
        str: Comprehensive summary of matching PDF files
    """
    all_docs = docs

    combined_text = " ".join([doc.page_content for doc in all_docs])

    summary_prompt = f"""
    Provide a comprehensive summary of all documents related to {query}, 
    covering the entire content of the knowledge base:

    {combined_text[:20000]}  # Limit to avoid token issues

    Key points to cover:
    - Overall context and scope
    - Main concepts and principles
    - Significant insights
    - Practical applications
    - Important technical details

    Detailed Summary:
    """
    
    try:
        summary = llm.invoke(summary_prompt).content
        return summary
    except Exception as e:
        return f"Error generating summary: {str(e)}"
    
@tool
def notes_file(query: str) -> str:
    """
    Makes notes of the PDF files related to the given query.
    
    Args:
        query (str): Search query to find relevant PDF files
    
    Returns:
        str: Notes from matching PDF files
    """
    all_docs = docs
    chunk_notes = []

    for doc in all_docs:
        chunk_prompt = f"""
        Provide a concise set of structured notes for the following document content
        related to {query}. Present the notes in clear bullet points.

        Document Excerpt:
        {doc.page_content[:5000]}

        Bullet-Point Notes:
        """
        try:
            note = llm.invoke(chunk_prompt).content
            chunk_notes.append(note)
        except Exception as e:
            chunk_notes.append(f"Error generating notes for a chunk: {str(e)}")

    combined_notes_prompt = f"""
    The following are notes from different document chunks related to {query}. 
    Summarize them into a single structured set of notes with key takeaways.

    Individual Notes:
    {"\n\n".join(chunk_notes)}

    Final Consolidated Notes:
    """

    try:
        final_notes = llm.invoke(combined_notes_prompt).content
        return final_notes
    except Exception as e:
        return f"Error generating final notes: {str(e)}"

    
tools = [add, subtract, multiply, divide, power, modulus, web_search, pdf_rag, summarize_file, notes_file]

In [6]:
memory = ConversationSummaryMemory(
    llm=llm,
    output_key="output"
)

prompt_template = PromptTemplate(
    template="""
You are an AI agent with access to these tools:
{tool_names}

Tools:
{tools}

When you need to use a tool, respond exactly in this format:

Thoughts: [Reasoning]
Action: [tool name]
Action Input:[tool input]

“If you take an action, do not produce a final answer. If you produce a final answer, do not produce an action.”

Thoughts: [Reasoning]

Final Answer: [answer here]

User Question: {user_input}

{agent_scratchpad}
""",
    input_variables=["tool_names", "tools", "user_input", "agent_scratchpad"]
)



agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template.partial(
        tool_names=", ".join([t.name for t in tools]),
        tools="\n".join([f"{t.name}: {t.description}" for t in tools])
    )
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

C:\Users\aliza\AppData\Local\Temp\ipykernel_27232\2542868426.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [18]:
response = agent_executor.invoke({"user_input": "can you make a summary on the convo in the pdf files"})
print(response)



> Entering new AgentExecutor chain...
Thoughts: The user is asking me to summarize the conversation in the PDF files. I will use the summarize_file tool to achieve this.

Action: summarize_file
Action Input: convoHere is a comprehensive summary of the provided text, covering the key points, main concepts, significant insights, practical applications, and important technical details:

**Overall Context and Scope:**
The text is an introduction to deep learning, a subfield of artificial intelligence (AI) that involves the use of neural networks to analyze and interpret data. The authors aim to provide a comprehensive overview of deep learning, its principles, and its applications.

**Main Concepts and Principles:**

1. **Artificial Intelligence (AI):** AI involves creating machines that can think and learn like humans.
2. **Machine Learning:** Machine learning is a type of AI that enables computers to learn from data without being explicitly programmed.
3. **Deep Learning:** Deep learni

KeyboardInterrupt: 